# Static database manupulation

In [1]:
import pandas as pd
import requests
import time
import json
from IPython.display import display

In [2]:
#Preliminary Parameters
file_name = 'pretty.csv'
fuel_cons = 'fuelcons.csv'
combined_databases = 'combined_databases.csv'

In [3]:
def split_data():
    pretty_df = pd.read_csv(file_name)
    #display(pretty_df)
    no_entries_df = pretty_df[pretty_df['fuel'] == 'NO DATA']
    old_entries_df = pretty_df[pretty_df['fuel'] != 'NO DATA']
    print(len(old_entries_df))
    no_entries_df = no_entries_df.drop(columns = ['fuel', 'co2', 'model'])
    return no_entries_df, old_entries_df

In [4]:
def match_iata_to_missing(no_data_entries_df, complete_codes_big_df):
    missing_with_iata_df = pd.merge(no_data_entries_df, complete_codes_big_df, on='icao', how='left')
    
    #input the missing models
    filtered_df = missing_with_iata_df[missing_with_iata_df['model'] != ""]
    model_df = filtered_df[['model', 'typecode']]
    model_df = model_df.drop_duplicates(subset=['typecode'])

    for index, row in missing_with_iata_df.iterrows():
        # Check if the model is empty
        if row['model'] == '':
            # Find the corresponding model from the second dataframe where typecode matches
            corresponding_model = model_df.loc[model_df['typecode'] == row['typecode'], 'model'].values
            # Update the model in the first dataframe if a corresponding model is found
            if len(corresponding_model) > 0:
                missing_with_iata_df.at[index, 'model'] = corresponding_model[0]

    return missing_with_iata_df

In [5]:
def calculate_fuel(distance, dist1, dist2, val1, val2):

    #using linear interpolation
    distance = float(distance)
    dist1 = float(dist1)
    dist2 = float(dist2)
    val1 = float(val1.iloc[0])
    val2 = float(val2.iloc[0])
    
    return round(val1 + ((val2 - val1) * (distance - dist1)) / (dist2 - dist1), 2)

In [6]:
def calculate_co2(fuel):
    return fuel * 3.16

In [7]:
def calculations(df):
    fuel_df = pd.read_csv(fuel_cons)

    #remove the d symbol out of the column names
    new_column_names = {}
    
    for column_name in fuel_df.columns:
        # Replace 'd' symbols with empty string
        new_column_name = column_name.replace('d', '')
        # Store the new column name in the dictionary
        new_column_names[column_name] = new_column_name

    # Rename the columns using the dictionary
    fuel_df = fuel_df.rename(columns=new_column_names)
    fuel_df.rename(columns={'Coe': 'code'}, inplace=True)
    df['fuel'] = 0
    df['co2'] = 0
    
    valid_iatacodes = df[df['iatacode'].isin(fuel_df['code'])]['iatacode']
    df_NaN= df[~df['iatacode'].isin(valid_iatacodes)]
    df = df[df['iatacode'].isin(valid_iatacodes)]

    #iterate through each row
    for index, row in df.iterrows():
        matching_row = fuel_df[fuel_df['code'] == row['iatacode']] ###
        distance = row['distance(km)']

        closest_columns = []
        
        # Find the thresholds
        lower_thresholds = [col for col in fuel_df.columns if col.isdigit() and int(col) <= distance]

        lower_bound = max(lower_thresholds) if lower_thresholds else None

        if lower_bound:
            closest_columns.append(lower_bound)
            #find upper bound
            column_index = fuel_df.columns.get_loc(lower_bound)
            if column_index < len(fuel_df.columns) - 1:
                # Get the name of the column to the right
                upper_bound = fuel_df.columns[column_index + 1]
                closest_columns.append(upper_bound)
            else:
                closest_columns.append(lower_bound)

        # Extract the corresponding values from the row
        
        if closest_columns[0] == 0:
            val1 = 0
        else:
            val1 = matching_row[closest_columns[0]]
        
        val2 = matching_row[closest_columns[1]]
        dist1 = closest_columns[0]
        dist2 = closest_columns[1]

        total_fuel = calculate_fuel(distance, dist1, dist2, val1, val2)
        co2 = calculate_co2(total_fuel)

        #put the values to the database
        df.loc[index, 'fuel'] = total_fuel
        df.loc[index, 'co2'] = co2
        
    print(len(df))
    print(len(df_NaN))
    
    return df, df_NaN
    

In [8]:
def combine_dataframes(old, new, no):

    new = new.copy()
    no = no.copy()
    new.drop(columns =['typecode', 'iatacode'], inplace = True)
    no.drop(columns =['typecode', 'iatacode'], inplace = True)
    #combined_df = pd.concat([old, new, no], ignore_index=True)
    combined_df = pd.concat([old, new], ignore_index=True)
    pd.set_option('display.max_rows', None)
    print(len(combined_df))
    combined_df.to_csv('processsed_data.csv')

In [9]:
def run_main():
    print("process started...")
    df_combined_databases = pd.read_csv(combined_databases, usecols=['icao','typecode', 'model', 'iatacode'])
    no_entries_df, old_entries_df = split_data()
    missing_with_iata_df = match_iata_to_missing(no_entries_df, df_combined_databases)
    no_entries_df, df_NaN = calculations(missing_with_iata_df)
    combine_dataframes(old_entries_df, no_entries_df, df_NaN)
    print("process finished")


In [10]:
run_main()

process started...
2791
569
581
3360
process finished
